In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from SALib.sample import saltelli, latin, fast_sampler, morris
from SALib.sample import sobol as ssobol
from SALib.analyze import sobol, fast, rbd_fast, delta, pawn
from SALib.analyze.sobol import analyze
from SALib.sample.sobol import sample
from SALib.test_functions import Ishigami
from tqdm import tqdm


In [13]:
# Define the ODE model
def model(y, t, params):
    k1, k2 = params
    dydt = [-k1 * y[0],
            k1 * y[0] - k2 * y[1],
            k2 * y[1]]
    return dydt

In [14]:
# Define parameter ranges
ode_problem = {
    'num_vars': 2,
    'names': ['k1', 'k2'],
    'bounds': [[0.1, 0.5], [0.1, 0.5]]
}

In [15]:
def add_data_to_large_df(time_point, small_df, large_df):
    # Initialize a dictionary to store data to be appended
    data = {'time_point': time_point}
    
    # Loop through each row in the small DataFrame and extract the needed values
    for idx in small_df.index:
        data[f'{idx}_delta'] = small_df.loc[idx, 'delta']
        data[f'{idx}_delta_conf'] = small_df.loc[idx, 'delta_conf']
        data[f'{idx}_S1'] = small_df.loc[idx, 'S1']
        data[f'{idx}_S1_conf'] = small_df.loc[idx, 'S1_conf']

    df_row = pd.DataFrame([data])
    # Append the new row to the larger DataFrame
    # large_df = large_df.append(df_row, ignoreIndex=True)
    large_df = pd.concat([large_df, df_row], ignore_index=True)
    return large_df

In [16]:
def generate_csv(problem, sample_size, time_points, parameter, output_name, file_path):

    # Create large dataframe
    df_large = pd.DataFrame(columns=['time_point'])

    # Generate samples
    param_values = ssobol.sample(problem, sample_size)

    # Define initial conditions and time points
    y0 = [10, 0, 0]
    t = np.linspace(0, 10, time_points)

    # Run the model for each sample
    output = np.empty([param_values.shape[0], t.size, 3])
    for i, params in enumerate(param_values):
        sol = odeint(model, y0, t, args=(params,))
        output[i] = sol
    
    for i, t_point in tqdm(enumerate(t)):
        if (i == 0):
            continue
    
        Si = delta.analyze(problem, param_values, output[:, i, parameter])
        df_large = add_data_to_large_df(t_point, Si.to_df(), df_large)

    df_large.to_csv((file_path + '/' + output_name + '.csv'), index = False)

In [17]:
generate_csv(ode_problem, 500, 100, 1, '1', '/Users/shivanimadhan/Documents/research/curve-fit/Experiments/Varying Sample Size/N_500')

/Users/shivanimadhan/anaconda3/envs/kinetics/lib/python3.11/site-packages/scipy/stats/_qmc.py:804: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  sample = self._random(n, workers=workers)
0it [00:00, ?it/s]/var/folders/zh/dpvqt3794n73s3md_1fs2wmr0000gn/T/ipykernel_8061/2738944218.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  large_df = pd.concat([large_df, df_row], ignore_index=True)
100it [03:23,  2.03s/it]


In [18]:
generate_csv(ode_problem, 500, 100, 0, '0', '/Users/shivanimadhan/Documents/research/curve-fit/Experiments/Varying Sample Size/N_500')
generate_csv(ode_problem, 100, 100, 0, '0', '/Users/shivanimadhan/Documents/research/curve-fit/Experiments/Varying Sample Size/N_100')
generate_csv(ode_problem, 100, 100, 1, '1', '/Users/shivanimadhan/Documents/research/curve-fit/Experiments/Varying Sample Size/N_100')
generate_csv(ode_problem, 100, 100, 2, '2', '/Users/shivanimadhan/Documents/research/curve-fit/Experiments/Varying Sample Size/N_100')
generate_csv(ode_problem, 1000, 100, 0, '0', '/Users/shivanimadhan/Documents/research/curve-fit/Experiments/Varying Sample Size/N_1000')
generate_csv(ode_problem, 1000, 100, 1, '1', '/Users/shivanimadhan/Documents/research/curve-fit/Experiments/Varying Sample Size/N_1000')
generate_csv(ode_problem, 1000, 100, 2, '2', '/Users/shivanimadhan/Documents/research/curve-fit/Experiments/Varying Sample Size/N_1000')

/Users/shivanimadhan/anaconda3/envs/kinetics/lib/python3.11/site-packages/scipy/stats/_qmc.py:804: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  sample = self._random(n, workers=workers)
0it [00:00, ?it/s]/var/folders/zh/dpvqt3794n73s3md_1fs2wmr0000gn/T/ipykernel_8061/2738944218.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  large_df = pd.concat([large_df, df_row], ignore_index=True)
100it [03:02,  1.82s/it]
/Users/shivanimadhan/anaconda3/envs/kinetics/lib/python3.11/site-packages/scipy/stats/_qmc.py:804: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  sample = self._random(n, workers=workers)
0it [00:00, ?it/s]/var/folders/zh/dpvqt3794n73s3md_1fs2wmr0000gn/T/

In [19]:
generate_csv(ode_problem, 500, 50, 0, '0', '/Users/shivanimadhan/Documents/research/curve-fit/Experiments/Varying Time Points/T_50')
generate_csv(ode_problem, 500, 50, 1, '1', '/Users/shivanimadhan/Documents/research/curve-fit/Experiments/Varying Time Points/T_50')
generate_csv(ode_problem, 500, 50, 2, '2', '/Users/shivanimadhan/Documents/research/curve-fit/Experiments/Varying Time Points/T_50')
generate_csv(ode_problem, 500, 100, 0, '0', '/Users/shivanimadhan/Documents/research/curve-fit/Experiments/Varying Time Points/T_100')
generate_csv(ode_problem, 500, 100, 1, '1', '/Users/shivanimadhan/Documents/research/curve-fit/Experiments/Varying Time Points/T_100')
generate_csv(ode_problem, 500, 100, 2, '2', '/Users/shivanimadhan/Documents/research/curve-fit/Experiments/Varying Time Points/T_100')
generate_csv(ode_problem, 500, 200, 0, '0', '/Users/shivanimadhan/Documents/research/curve-fit/Experiments/Varying Time Points/T_200')
generate_csv(ode_problem, 500, 200, 1, '1', '/Users/shivanimadhan/Documents/research/curve-fit/Experiments/Varying Time Points/T_200')
generate_csv(ode_problem, 500, 200, 2, '2', '/Users/shivanimadhan/Documents/research/curve-fit/Experiments/Varying Time Points/T_200')

/Users/shivanimadhan/anaconda3/envs/kinetics/lib/python3.11/site-packages/scipy/stats/_qmc.py:804: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  sample = self._random(n, workers=workers)
0it [00:00, ?it/s]/var/folders/zh/dpvqt3794n73s3md_1fs2wmr0000gn/T/ipykernel_8061/2738944218.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  large_df = pd.concat([large_df, df_row], ignore_index=True)
50it [01:20,  1.60s/it]
/Users/shivanimadhan/anaconda3/envs/kinetics/lib/python3.11/site-packages/scipy/stats/_qmc.py:804: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  sample = self._random(n, workers=workers)
0it [00:00, ?it/s]/var/folders/zh/dpvqt3794n73s3md_1fs2wmr0000gn/T/i